importing Libraries

In [1]:
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
import keras
from keras.layers import Conv2D,Activation,Dense,Input, MaxPooling2D , ZeroPadding2D,Flatten
from keras.models import Model, load_model,Sequential

Using TensorFlow backend.


Loading a pre-trained model

In [2]:
model=load_model('facenet_keras.h5')

C:\Users\YASH\Anaconda3\envs\Tensorflow-gpu\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


changing the dimension of input layer

In [3]:
model.layers[0].batch_input_shape=(None,200,200,3)
model.layers[0].batch_input_shape

(None, 200, 200, 3)

In [4]:
# rebuild model architecture by exporting and importing via json
new_model1 = keras.models.model_from_json(model.to_json())

    # copy weights from old model to new one
for layer in new_model1.layers:
    try:
        layer.set_weights(model.get_layer(name=layer.name).get_weights())
        print("Loaded layer {}".format(layer.name))
    except:
        print("Could not transfer weights for layer {}".format(layer.name))

Loaded layer input_1
Loaded layer Conv2d_1a_3x3
Loaded layer Conv2d_1a_3x3_BatchNorm
Loaded layer Conv2d_1a_3x3_Activation
Loaded layer Conv2d_2a_3x3
Loaded layer Conv2d_2a_3x3_BatchNorm
Loaded layer Conv2d_2a_3x3_Activation
Loaded layer Conv2d_2b_3x3
Loaded layer Conv2d_2b_3x3_BatchNorm
Loaded layer Conv2d_2b_3x3_Activation
Loaded layer MaxPool_3a_3x3
Loaded layer Conv2d_3b_1x1
Loaded layer Conv2d_3b_1x1_BatchNorm
Loaded layer Conv2d_3b_1x1_Activation
Loaded layer Conv2d_4a_3x3
Loaded layer Conv2d_4a_3x3_BatchNorm
Loaded layer Conv2d_4a_3x3_Activation
Loaded layer Conv2d_4b_3x3
Loaded layer Conv2d_4b_3x3_BatchNorm
Loaded layer Conv2d_4b_3x3_Activation
Loaded layer Block35_1_Branch_2_Conv2d_0a_1x1
Loaded layer Block35_1_Branch_2_Conv2d_0a_1x1_BatchNorm
Loaded layer Block35_1_Branch_2_Conv2d_0a_1x1_Activation
Loaded layer Block35_1_Branch_1_Conv2d_0a_1x1
Loaded layer Block35_1_Branch_2_Conv2d_0b_3x3
Loaded layer Block35_1_Branch_1_Conv2d_0a_1x1_BatchNorm
Loaded layer Block35_1_Branch_2_

In [5]:
new_model1.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 99, 99, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 99, 99, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 99, 99, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

changing the model according to requirement

In [6]:
#transfer_layer points to the dropout layer
transfer_layer = new_model1.get_layer('Dropout')

In [7]:
#defining model with output as the transfer_layer
conv_model = Model(inputs=new_model1.input,
                   outputs=transfer_layer.output)

In [8]:
#converting the model into sequential model
new_model = Sequential()
new_model.add(conv_model)

In [9]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 1792)              22578384  
Total params: 22,578,384
Trainable params: 22,549,808
Non-trainable params: 28,576
_________________________________________________________________


In [10]:
new_model.add(Dense(388)) # adding a layer 

In [11]:
#function to print whether our layers are trainable or not 
def print_layer_trainable():
    for layer in conv_model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))
        
# making all the layer untrainable
for layer in conv_model.layers:
    layer.trainable = False

In [13]:
# making some layer trainable
for layer in conv_model.layers:
    # Boolean whether this layer is trainable.
    trainable = ('Dropout' in layer.name or 'AvgPool' in layer.name or 'Block8' in layer.name)
    
    # Set the layer's bool.
    layer.trainable = trainable

In [14]:
print_layer_trainable()

False:	input_1
False:	Conv2d_1a_3x3
False:	Conv2d_1a_3x3_BatchNorm
False:	Conv2d_1a_3x3_Activation
False:	Conv2d_2a_3x3
False:	Conv2d_2a_3x3_BatchNorm
False:	Conv2d_2a_3x3_Activation
False:	Conv2d_2b_3x3
False:	Conv2d_2b_3x3_BatchNorm
False:	Conv2d_2b_3x3_Activation
False:	MaxPool_3a_3x3
False:	Conv2d_3b_1x1
False:	Conv2d_3b_1x1_BatchNorm
False:	Conv2d_3b_1x1_Activation
False:	Conv2d_4a_3x3
False:	Conv2d_4a_3x3_BatchNorm
False:	Conv2d_4a_3x3_Activation
False:	Conv2d_4b_3x3
False:	Conv2d_4b_3x3_BatchNorm
False:	Conv2d_4b_3x3_Activation
False:	Block35_1_Branch_2_Conv2d_0a_1x1
False:	Block35_1_Branch_2_Conv2d_0a_1x1_BatchNorm
False:	Block35_1_Branch_2_Conv2d_0a_1x1_Activation
False:	Block35_1_Branch_1_Conv2d_0a_1x1
False:	Block35_1_Branch_2_Conv2d_0b_3x3
False:	Block35_1_Branch_1_Conv2d_0a_1x1_BatchNorm
False:	Block35_1_Branch_2_Conv2d_0b_3x3_BatchNorm
False:	Block35_1_Branch_1_Conv2d_0a_1x1_Activation
False:	Block35_1_Branch_2_Conv2d_0b_3x3_Activation
False:	Block35_1_Branch_0_Conv2d_1x1

importing the data

In [17]:
a=np.load('out.npy')                        # output labels
b=np.load('img.npy')/255                    # input images
out=a[:2000,:,:]
out=np.reshape(out,[2000,388])
testout=a[2000: ,:,:]
testout=np.reshape(testout,[330,388])
testimg=b[2000:,:,:,:]
img=b[:2000,:,:,:]

In [18]:
opt= keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)

new_model.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])

training the model

In [19]:
new_model.fit(img,out,epochs=500, batch_size=32,verbose=1)

Epoch 1/500
2000/2000 [==============================] - 40s 20ms/step - loss: 10090.8093 - accuracy: 0.0020
Epoch 2/500
2000/2000 [==============================] - 18s 9ms/step - loss: 4452.0094 - accuracy: 0.0035
Epoch 3/500
2000/2000 [==============================] - 18s 9ms/step - loss: 487.2952 - accuracy: 0.0265
Epoch 4/500
2000/2000 [==============================] - 18s 9ms/step - loss: 201.8123 - accuracy: 0.0580
Epoch 5/500
2000/2000 [==============================] - 18s 9ms/step - loss: 188.9745 - accuracy: 0.0770
Epoch 6/500
2000/2000 [==============================] - 18s 9ms/step - loss: 187.1076 - accuracy: 0.0735
Epoch 7/500
2000/2000 [==============================] - 18s 9ms/step - loss: 186.2481 - accuracy: 0.0725
Epoch 8/500
2000/2000 [==============================] - 18s 9ms/step - loss: 184.2776 - accuracy: 0.0915
Epoch 9/500
2000/2000 [==============================] - 18s 9ms/step - loss: 183.4084 - accuracy: 0.0935
Epoch 10/500
2000/2000 [==================

In [20]:
new_model.save('face_model_pre_block8.h5')

Testing the model on images

In [106]:
path = 'D:\\Downloads\\New folder\\annotation\\' + '100' +'.txt'
df = pd.read_table(path, delim_whitespace=True, names=('A', 'B','C'))
    
img_name= df.iloc[0,0]
img_path = 'C:\\Users\\YASH\\face feature detection\\helen_1\\' + img_name +'.jpg'
img=cv2.imread(img_path)

shape1 = img.shape
img=cv2.resize(img,(200,200))
img1 =img/255
img1 = np.reshape(img1,[1,200,200,3])
h=new_model.predict(img1)
h=h
h=h.round()
h=h.astype(int)
x=h[:,0:194]
y=h[:,194:]
img = np.reshape(img,[200,200,3])
for i in range(0,194):
    img[y[:,i],x[:,i],:]=255
    
cv2.imshow("s",img)
cv2.waitKey(10000)
cv2.destroyAllWindows()